In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

tf.set_random_seed(777)

def data_standardization(x):
    x_np = np.asarray(x)
    return (x_np - x_np.mean()) / x_np.std()

def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np - x_np.min()) / (x_np.max() - x_np.min() + 1e-7)

def reverse_min_max_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()
 
 

input_data_column_cnt = 6
output_data_column_cnt = 1
 
seq_length = 3
rnn_cell_hidden_dim = 20
forget_bias = 1.0
num_stacked_layers = 1
keep_prob = 0.8
 
epoch_num = 1000
learning_rate = 0.01
 

stock_file_name = 'TSLA.csv'
encoding = 'utf-8'
names = ['Date','Open','High','Low','Close','Adj Close','Volume']
raw_dataframe = pd.read_csv(stock_file_name, names=names, encoding=encoding)
raw_dataframe.info()
 

del raw_dataframe['Date']
 
stock_info = raw_dataframe.values[1:].astype(np.float)
print("stock_info.shape: ", stock_info.shape)
print("stock_info[0]: ", stock_info[0])
 

price = stock_info[:,:-1]
norm_price = min_max_scaling(price)
print("price.shape: ", price.shape)
print("price[0]: ", price[0])
print("norm_price[0]: ", norm_price[0])
print("="*100)

volume = stock_info[:,-1:]
norm_volume = min_max_scaling(volume)
print("volume.shape: ", volume.shape)
print("volume[0]: ", volume[0])
print("norm_volume[0]: ", norm_volume[0])
print("="*100)
 
x = np.concatenate((norm_price, norm_volume), axis=1)
print("x.shape: ", x.shape)
print("x[0]: ", x[0])
print("x[-1]: ", x[-1])
print("="*100)
 
y = x[:, [-2]]
print("y[0]: ",y[0])
print("y[-1]: ",y[-1])
 
 
dataX = []
dataY = []
 
for i in range(0, len(y) - seq_length):
    _x = x[i : i+seq_length]
    _y = y[i + seq_length]
    if i is 0:
        print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)
 
 
train_size = int(len(dataY) * 0.7)

test_size = len(dataY) - train_size
 

trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])
 

testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])
 

X = tf.placeholder(tf.float32, [None, seq_length, input_data_column_cnt])
print("X: ", X)
Y = tf.placeholder(tf.float32, [None, 1])
print("Y: ", Y)
 

targets = tf.placeholder(tf.float32, [None, 1])
print("targets: ", targets)
 
predictions = tf.placeholder(tf.float32, [None, 1])
print("predictions: ", predictions)
 
 

def lstm_cell():

    # num_units
    # forget_bias:  to the biases of the forget gate 
    #              (default: 1)  in order to reduce the scale of forgetting in the beginning of the training.
    # state_is_tuple: True ==> accepted and returned states are 2-tuples of the c_state and m_state.
    # state_is_tuple: False ==> they are concatenated along the column axis.
    cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units=rnn_cell_hidden_dim, 
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        cell = tf.compat.v1.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell
 

stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
multi_cells = tf.compat.v1.nn.rnn_cell.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()
 

hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("hypothesis: ", hypothesis)
 

hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], output_data_column_cnt, activation_fn=tf.identity)
 
 

loss = tf.reduce_sum(tf.square(hypothesis - Y))

optimizer = tf.train.AdamOptimizer(learning_rate)
 
train = optimizer.minimize(loss)
 

rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))
 
 
train_error_summary = []
test_error_summary = []
test_predict = ''
 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
 

start_time = datetime.datetime.now()
print('Start learning...')
for epoch in range(epoch_num):
    _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
    if ((epoch+1) % 100 == 0) or (epoch == epoch_num-1):

        train_predict = sess.run(hypothesis, feed_dict={X: trainX})
        train_error = sess.run(rmse, feed_dict={targets: trainY, predictions: train_predict})
        train_error_summary.append(train_error)
 

        test_predict = sess.run(hypothesis, feed_dict={X: testX})
        test_error = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
        test_error_summary.append(test_error)
        

        print("epoch: {}, train_error(A): {}, test_error(B): {}, B-A: {}".format(epoch+1, train_error, test_error, test_error-train_error))
        
end_time = datetime.datetime.now()
elapsed_time = end_time - start_time
print('elapsed_time:',elapsed_time)
print('elapsed_time per epoch:',elapsed_time/epoch_num)
 

print('input_data_column_cnt:', input_data_column_cnt, end='')
print(',output_data_column_cnt:', output_data_column_cnt, end='')
 
print(',seq_length:', seq_length, end='')
print(',rnn_cell_hidden_dim:', rnn_cell_hidden_dim, end='')
print(',forget_bias:', forget_bias, end='')
print(',num_stacked_layers:', num_stacked_layers, end='')
print(',keep_prob:', keep_prob, end='')
 
print(',epoch_num:', epoch_num, end='')
print(',learning_rate:', learning_rate, end='')
 
print(',train_error:', train_error_summary[-1], end='')
print(',test_error:', test_error_summary[-1], end='')
print(',min_test_error:', np.min(test_error_summary))
 

plt.figure(1)
plt.plot(train_error_summary, 'gold')
plt.plot(test_error_summary, 'b')
plt.xlabel('Epoch(x100)')
plt.ylabel('Root Mean Square Error')
 
plt.figure(2)
plt.plot(testY, 'r')
plt.plot(test_predict, 'b')
plt.xlabel('Time Period')
plt.ylabel('Stock Price')
plt.show()
 
 

recent_data = np.array([x[len(x)-seq_length : ]])
print("recent_data.shape:", recent_data.shape)
print("recent_data:", recent_data)
 

test_predict = sess.run(hypothesis, feed_dict={X: recent_data})
 
print("test_predict", test_predict[0])
test_predict = reverse_min_max_scaling(price,test_predict)
print("Tomorrow's stock price", test_predict[0])

Instructions for updating:
non-resource variables are not supported in the long term
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       400 non-null    object
 1   Open       400 non-null    object
 2   High       400 non-null    object
 3   Low        400 non-null    object
 4   Close      400 non-null    object
 5   Adj Close  400 non-null    object
 6   Volume     400 non-null    object
dtypes: object(7)
memory usage: 22.0+ KB
stock_info.shape:  (399, 6)
stock_info[0]:  [6.1220001e+01 6.3026001e+01 5.9759998e+01 6.2023998e+01 6.2023998e+01
 5.8293000e+07]
price.shape:  (399, 5)
price[0]:  [61.220001 63.026001 59.759998 62.023998 62.023998]
norm_price[0]:  [0.07979605 0.08537702 0.0752843  0.08228059 0.08228059]
volume.shape:  (399, 1)
volume[0]:  [58293000.]
norm_volume[0]:  [0.15721732]
x.shape:  (399, 6)
x[0]:  [0.07979605 0.08537702 0.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


AttributeError: module 'tensorflow.compat.v1' has no attribute 'contrib'

In [20]:
!pip install tensorflow-addons

     |████████████████████████████████| 703 kB 5.1 MB/s eta 0:00:01


In [22]:
!pip install finance-datareader

     |████████████████████████████████| 5.5 MB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 5.7 MB/s  eta 0:00:01
